[kernel #1](https://www.kaggle.com/sentdex/dogs-vs-cats-redux-kernels-edition/full-classification-example-with-convnet)

[kernel #2](https://www.kaggle.com/jeffd23/dogs-vs-cats-redux-kernels-edition/catdognet-keras-convnet-starter)

[kernel #3](https://www.kaggle.com/cgallay/dogs-vs-cats-redux-kernels-edition/cat-dog-notebook)

https://pythonprogramming.net/cnn-tensorflow-convolutional-nerual-network-machine-learning-tutorial/

https://pythonprogramming.net/tflearn-machine-learning-tutorial/

In [1]:
import cv2
import os
from tqdm import tqdm # progress bar for long operations

import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

from sklearn.model_selection import train_test_split

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
image_shape = (128, 128)
LR = 0.001

In [3]:
Y_train = []
X_train = []

for filename in tqdm(os.listdir("train")):
    Y_train.append(int(filename.startswith("cat")))
    full_path = os.path.join("train", filename)
    image = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, image_shape)
    X_train.append([np.array(image).reshape(image_shape).astype(np.float32)])

Y_train = np.array(Y_train)
Y_train = np.concatenate((Y_train.reshape(-1, 1), 1 - Y_train.reshape(-1, 1)), axis=1)
X_train = np.concatenate(tuple(X_train), axis=0).reshape(-1, image_shape[0], image_shape[1], 1)

np.save("X_train", X_train)
np.save("Y_train", Y_train)

X_test = []

for filename in tqdm(os.listdir("test")):
    full_path = os.path.join("test", filename)
    image = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, image_shape)
    X_test.append([np.array(image).reshape(image_shape).astype(np.float32)])

X_test = np.concatenate(tuple(X_test), axis=0).reshape(-1, image_shape[0], image_shape[1], 1)

np.save("X_test", X_test)

100%|███████████████████████████████████████████████████████████████████████████| 12500/12500 [00:13<00:00, 939.81it/s]


In [2]:
# reload arrays if shape is not changed

X_train = np.load("X_train.npy")
Y_train = np.load("Y_train.npy")

X_test = np.load("X_test.npy")

In [4]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1)

In [5]:
convnet = input_data(shape=[None, image_shape[0], image_shape[1], 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

In [6]:
model.fit({'input': X_train}, {'targets': Y_train}, n_epoch=4, validation_set=({'input': X_val}, {'targets': Y_val}), snapshot_step=500, show_metric=True)

Training Step: 1407  | total loss: 0.37817 | time: 20.520s
| Adam | epoch: 004 | loss: 0.37817 - acc: 0.8288 -- iter: 22464/22500
Training Step: 1408  | total loss: 0.36488 | time: 21.603s
| Adam | epoch: 004 | loss: 0.36488 - acc: 0.8382 | val_loss: 0.40325 - val_acc: 0.8268 -- iter: 22500/22500
--
